In [1]:
import requests
import pandas as pd
import numpy as np
from IPython.display import display

In [2]:
# used to get lap data on a desired car and no of laps
def get_lapData(session, car_no, start, end):
    print(f'Lap data for car No. {car_no} is on its way!!')
    data_list = []

    if session == '':
        session = 'latest'
    
    for i in range(start, end):
        response = requests.get(f'https://api.openf1.org/v1/laps?session_key={session}&driver_number={car_no}&lap_number={i}')

        if response.status_code != 200:
            return 'API request error now. Try again!'
        
        data = response.json()
        
        if len(data) > 0:
            data_list.append(data[0])

    return data_list

# function used to clean lap data
def clean_lapData(df):
    df.drop(columns=['date_start', 'meeting_key', 'session_key', 'date_start'])
    df.rename(columns={"duration_sector_1": "S1", "duration_sector_2": "S2", "duration_sector_3" : 'S3'}, inplace=True)
    return df


In [3]:
# used to convert the list data into a df
def to_df(data):
    return pd.DataFrame(data)

# used to add data to csv
def into_csv(df):
    return df.to_csv('Data/SPA.csv', index=False)
    

In [4]:
# used to get info on stints
def get_stint(session, car_no, tyre_age):
    print(f'Stint data for car No. {car_no} is incoming!!')

    data_list = []

    if session == '':
        session = 'latest'
    
    response = requests.get(f'https://api.openf1.org/v1/stints?session_key={session}&tyre_age_at_start>={tyre_age}')

    if response.status_code != 200:
        return 'API request error currently. Review info and try again!!'
    
    data = response.json()

    for i in range(0, len(data)):
        if data[i]['driver_number'] == car_no: 
            data_list.append(data[i])
            
    return data_list

# function used to clean stint data
def clean_stintData(df): 
    df.drop(columns=['date_start', 'meeting_key', 'session_key'])
    return df

In [5]:
# used to get data relating to car performance 
def get_carData(session, car_no):
    print(f'Car data for car No. {car_no} is on its way!!')

    data_list = []

    if session == '':
        session = 'latest'

    response = requests.get(f'https://api.openf1.org/v1/car_data?driver_number={car_no}&session_key={session}&speed>=330')

    if response.status_code != 200:
        return 'API request error currently. Review info and try again!!'
    
    data = response.json()

    for i in range(0, len(data)):
        if data[i]['driver_number'] == car_no: 
            data_list.append(data[i])
            
    return data_list

In [26]:
# used to get session key and correlate fruther to retrive lap data
def get_session(country, year, type):
    # type options: Race, Sprint, Qualifying, Pratice (1, 2, 3)
    print(f'Getting session info for {country} {year}, {type}!!')

    response = requests.get(f'https://api.openf1.org/v1/sessions?country_name={country}&session_name={type}&year={year}')

    if response.status_code != 200:
        return 'API request error currently. Review info and try again!!'

    data = response.json()

    data_list = {
        'session': data[0]['session_key'],
        'meeting': data[0]['meeting_key']
    }

    return data_list

def clean_session(df):
    df = df.drop(columns=['meeting_key'])
    return df

In [7]:
# Full code example in use, i.e. 1

# 1. State the session you want to get information for!
session  = get_session('Belgium', '2024', 'Race')

# 2. Get stint data for the race, with session key just found...
stint_data = get_stint(session['session'], 44, 0)

# 3. Unpack the desired stint data to retrive the laps
start = stint_data[0]['lap_start']
end = stint_data[0]['lap_end'] + 1

# 4. feed the stint lap data, into the full get lap data method
lap_data = get_lapData(session['session'], 44, start + 1, end)  # +1 is needed to avoid api error

# 5. conver the lap data into a df for futher anylsis
lap_df = to_df(lap_data)

# 5.5 apply the needed methods to clean the lap df
lap_df = clean_lapData(lap_df)

lap_df

Getting session info for Belgium 2024, Race!!
Stint data for car No. 44 is incoming!!
Lap data for car No. 44 is on its way!!


,meeting_key,session_key,driver_number,i1_speed,i2_speed,st_speed,date_start,lap_duration,is_pit_out_lap,S1,S2,S3,segments_sector_1,segments_sector_2,segments_sector_3,lap_number
0,1242,9574,44,329.0,202,307.0,2024-07-28T13:05:45.881000+00:00,109.849,False,31.284,49.211,29.354,"[2049, 2049, 2048, 2049, 2048, 2048, 2049, 2049]","[2049, 2049, 2051, 2049, 2048, 2049, 2049, 204...","[2048, 2048, 2048, 2048, 2048, 2051, 0]",2
1,1242,9574,44,326.0,204,310.0,2024-07-28T13:07:35.621000+00:00,109.109,False,31.215,48.486,29.408,"[None, 2048, 2049, 2049, 2049, 2049, 2049, 2048]","[2049, 2049, 2048, 2048, 2049, 2049, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2051, 2048]",3
2,1242,9574,44,305.0,203,305.0,2024-07-28T13:09:24.787000+00:00,109.808,False,31.754,48.605,29.449,"[2048, 2051, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2051, 2051, 2048, 2048, 2051, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",4
3,1242,9574,44,307.0,201,307.0,2024-07-28T13:11:14.724000+00:00,109.376,False,31.592,48.320,29.464,"[2049, 2051, 2048, 2048, 2048, 2048, 2048, 2048]","[2049, 2048, 2051, 2048, 2048, 2051, 2048, 204...","[2049, 2048, 2048, 2048, 2048, 2049, 2048]",5
4,1242,9574,44,308.0,204,NaN,2024-07-28T13:13:04.006000+00:00,109.400,False,31.618,48.355,29.427,"[None, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2051, 2048, 2048, 2051, 2049, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",6
5,1242,9574,44,309.0,202,306.0,2024-07-28T13:14:53.395000+00:00,109.357,False,31.662,48.124,29.571,"[2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2049, 2051, 2048, 2049, 2048, 2048, 2049, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",7
6,1242,9574,44,NaN,200,NaN,2024-07-28T13:16:42.767000+00:00,109.666,False,31.646,48.515,29.505,"[None, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2049, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",8
7,1242,9574,44,NaN,200,307.0,2024-07-28T13:18:32.516000+00:00,109.704,False,31.616,48.572,29.516,"[None, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",9
8,1242,9574,44,NaN,200,NaN,2024-07-28T13:20:22.216000+00:00,109.951,False,31.625,48.752,29.574,"[None, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2049, 2064]",10
9,1242,9574,44,311.0,202,306.0,2024-07-28T13:22:12.078000+00:00,114.328,False,31.618,48.726,33.984,"[None, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2049, 2049, 2049, 2049, 2048, 2051]",11


In [ ]:
# Full code example in use, i.e. 2
data = []
car_no = [44, 63]

# 1. State the session you want to get information for!
session  = get_session('Belgium', '2024', 'Race')

for car in car_no:
    # 2. Get stint data for the race, with session key just found...
    stint_data = get_stint(session['session'], car, 0)

    # 3. Unpack the desired stint data to retrive the laps
    start = stint_data[len(stint_data) - 1]['lap_start']
    end = stint_data[len(stint_data) - 1]['lap_end'] + 1

    # 4. feed the stint lap data, into the full get lap data method
    lap_data = get_lapData(session['session'], car, start, end)  # +1 is needed to avoid api error, for initial stint

    # 5. conver the lap data into a df for futher anylsis
    lap_df = to_df(lap_data)

    # 5.5 apply the needed methods to clean the lap df
    lap_df = clean_lapData(lap_df)

    # 6. add it to a csv
    into_csv(lap_df)

    # 7. display the df's
    display(lap_df)

In [44]:
# imports for machine learning and k-means clustering
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [62]:
# full code example using machine learning (pt.1)
# Step 1: Get the session id pratice
prac_sessions = [1, 2, 3]
car_no = 44
session_list = []

for s in prac_sessions:
    session = get_session('Belgium', '2024', f'Practice {s}')
    session_list.append(session['session'])

# Step 2: get the stint data for the sessions found
stint_data = []
for s in session_list:
    stint = get_stint(s, car_no, 0)
    df = to_df(stint)
    stint_data.append(df)

stint = pd.concat(stint_data, ignore_index=True)
stint = clean_session(stint[stint['compound'] != 'INTERMEDIATE'])

# refresh the session list to note any changes after cleaning
session_list =  stint['session_key'].unique().tolist()

# Step 3: get the lap data for each stint in the sessions found
data = []
for s in session_list:
    # Filter the DataFrame for the current session
    session_df = stint.loc[stint['session_key'] == s]
    
    # Get the minimum lap start and maximum lap end for the filtered session
    start = session_df['lap_start'].min()
    end = session_df['lap_end'].max()

    # Fetch lap data using the start and end values
    lap = get_lapData(s, car_no, start, end)
    lap = to_df(lap)
    lap = lap[lap['is_pit_out_lap']==False]

    data.append(lap)

# Step 4: concat all togther to make a larger df, for pratice
if data:
    practice_data = pd.concat(data, ignore_index=True)

display(stint)




Getting session info for Belgium 2024, Practice 1!!
Getting session info for Belgium 2024, Practice 2!!
Getting session info for Belgium 2024, Practice 3!!
Stint data for car No. 44 is incoming!!
Stint data for car No. 44 is incoming!!
Stint data for car No. 44 is incoming!!
Lap data for car No. 44 is on its way!!
Lap data for car No. 44 is on its way!!


,session_key,stint_number,driver_number,lap_start,lap_end,compound,tyre_age_at_start
0,9567,1,44,1,7,MEDIUM,0
1,9567,2,44,8,11,SOFT,0
2,9567,3,44,12,16,MEDIUM,6
3,9567,4,44,17,22,MEDIUM,11
4,9568,1,44,1,9,SOFT,0
5,9568,2,44,10,14,SOFT,0
6,9568,3,44,15,26,SOFT,5


In [63]:
display(practice_data)

,meeting_key,session_key,driver_number,i1_speed,i2_speed,st_speed,date_start,lap_duration,is_pit_out_lap,duration_sector_1,duration_sector_2,duration_sector_3,segments_sector_1,segments_sector_2,segments_sector_3,lap_number
0,1242,9567,44,332.0,211,304.0,2024-07-26T11:32:23.408000+00:00,105.888,False,30.929,46.331,28.628,"[2049, 2049, 2049, 2049, 2049, 2049, 2049, 2049]","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",2
1,1242,9567,44,105.0,102,107.0,2024-07-26T11:34:09.270000+00:00,240.494,False,71.753,112.436,56.305,"[2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2051]",3
2,1242,9567,44,340.0,77,308.0,2024-07-26T11:38:09.842000+00:00,141.107,False,30.811,70.479,39.817,"[None, 2048, 2049, 2049, 2049, 2049, 2049, 2049]","[2051, 2048, 2049, 2048, 2048, 2048, 2048, 204...","[2048, 2049, 2048, 2048, 2048, 2048, 2048]",4
3,1242,9567,44,331.0,204,306.0,2024-07-26T11:40:30.834000+00:00,105.207,False,30.868,45.732,28.607,"[2049, 2048, 2049, 2048, 2048, 2048, 2048, 2048]","[2048, 2049, 2049, 2049, 2048, 2049, 2049, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2064]",5
4,1242,9567,44,167.0,196,269.0,2024-07-26T11:42:16.089000+00:00,129.863,False,38.588,56.204,35.071,"[2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2049]",6
5,1242,9567,44,333.0,210,308.0,2024-07-26T12:00:09.636000+00:00,104.279,False,30.588,45.136,28.555,"[2049, 2049, 2049, 2048, 2048, 2048, 2048, 2048]","[2051, 2048, 2048, 2048, 2049, 2049, 2049, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",8
6,1242,9567,44,141.0,97,164.0,2024-07-26T12:01:53.804000+00:00,247.853,False,54.673,113.760,79.420,"[2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2064]",9
7,1242,9567,44,332.0,197,304.0,2024-07-26T12:06:01.727000+00:00,125.221,False,30.827,59.859,34.535,"[None, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2049, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",10
8,1242,9567,44,316.0,203,295.0,2024-07-26T12:15:40.847000+00:00,111.263,False,31.772,50.151,29.340,"[2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",12
9,1242,9567,44,318.0,201,297.0,2024-07-26T12:17:32.155000+00:00,109.232,False,31.808,48.116,29.308,"[None, 2048, 2048, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048, 2048, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2048]",13


In [56]:
# full code example using machine learning (pt.2)
# Step 5: get the race data for comparison
race_data = []
session  = get_session('Belgium', '2024', 'Race')
print('hi')
session
race_data = get_lapData(session=session[0], car_no=car_no, start=1, end=44)

race_data

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().